In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5
from DataInfo import ISU_list_MBRN_sum_top,MBRN_sum_top

In [2]:
# TickDataname='GDF2022_5_4_8_25.csv'
TickDataname='GDF2022_7_31_21_9.csv'
TickGDF=pd.read_csv('/Data/LOBData/CollectedTickData/'+TickDataname)
Dataname='norm6featNewPrc'

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

In [3]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [4]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [5]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']
minmaxnormlist=[]
minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격']

In [6]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [7]:
Train_df[loglist]=np.log(Train_df[loglist])

In [8]:
Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()-Train_df[minmaxnormlist].min())

In [9]:
# ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickDataname='GDF_ORD2022_8_1_17_36.csv'
ORDTickGDF=pd.read_csv('/Data/LOBData/CollectedTickData/'+ORDTickDataname)

In [10]:
ORDTickGDF

,Unnamed: 0,ORD_DD,ISU_CD,MBR_NO,BRN_NO,ASKBID_TP_CD,MODCANCL_TP_CD,ORD_ACPT_TM,ORD_QTY,ORD_PRC,TM_GP,ORD_VOL
0,0,20170601,KR7001840008,2,1,1,1,NaN,501,0.0,1.0,0.0
1,1,20170601,KR7001840008,2,1,2,1,NaN,70,7040.0,9.0,492800.0
2,2,20170601,KR7001840008,2,1,1,1,NaN,70,6980.0,17.0,488600.0
3,3,20170601,KR7001840008,2,28,2,1,NaN,300,7030.0,9.0,2109000.0
4,4,20170601,KR7001840008,2,28,1,1,NaN,400,8000.0,15.0,3200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11934373,121062,20170929,KR7265520007,56,17990,1,1,NaN,300,39800.0,13.0,11940000.0
11934374,121063,20170929,KR7265520007,56,17990,2,1,NaN,25,39800.0,32.0,995000.0
11934375,121064,20170929,KR7265520007,56,17990,1,1,NaN,700,39637.5,37.0,27755000.0
11934376,121065,20170929,KR7265520007,56,17990,1,3,NaN,200,39650.0,37.0,7930000.0


In [11]:
ORDTickGDF.columns

Index(['Unnamed: 0', 'ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO', 'ASKBID_TP_CD',
       'MODCANCL_TP_CD', 'ORD_ACPT_TM', 'ORD_QTY', 'ORD_PRC', 'TM_GP',
       'ORD_VOL'],
      dtype='object')

In [12]:
ORDTickGDF['ORD_PRC']

0               0.0
1            7040.0
2            6980.0
3            7030.0
4            8000.0
             ...   
11934373    39800.0
11934374    39800.0
11934375    39637.5
11934376    39650.0
11934377    40300.0
Name: ORD_PRC, Length: 11934378, dtype: float64

In [13]:
ORDTickGDF['ASKBID_TP_CD']

0           1
1           2
2           1
3           2
4           1
           ..
11934373    1
11934374    2
11934375    1
11934376    1
11934377    1
Name: ASKBID_TP_CD, Length: 11934378, dtype: int64

In [14]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)#'norm6featNew'
OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*OTGDF['ORD_PRC']#'norm6featNewPrc'
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [15]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [16]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [17]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [18]:
feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

In [19]:
feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]

In [20]:
mbrnlist = MBRN_sum_top

In [21]:
Train_df[feat_cols[:-1]]

,매도5단계호가합계잔량,매수5단계호가합계잔량,매도10단계호가합계잔량,매수10단계호가합계잔량,매도총호가잔량,매수총호가잔량,고가,저가,시가,직전체결가격
0,13212.680851,14687.278723,26249.319149,27638.774468,0.024618,0.012885,0.306272,0.887197,8.832004,0.535697
1,20547.272222,13364.017778,39778.534444,29118.977778,0.034666,0.015497,0.350581,0.884147,8.832004,0.560337
2,12931.410204,17491.644898,41123.810204,40261.324490,0.042802,0.016714,0.357820,0.884147,8.832004,0.548632
3,6224.275748,23620.970100,20487.966777,49493.837209,0.046580,0.016553,0.357820,0.884147,8.832004,0.535904
4,11784.360577,21477.052885,29705.149038,51873.399038,0.050092,0.017151,0.357820,0.884147,8.832004,0.537080
...,...,...,...,...,...,...,...,...,...,...
331495,6523.719298,3714.280702,13598.140351,5830.263158,0.004551,0.001800,0.327127,0.905415,10.573878,0.540597
331496,7433.955224,3459.641791,15011.597015,5328.641791,0.004633,0.001702,0.327127,0.905415,10.573878,0.539718
331497,7322.410256,3585.794872,16086.705128,4686.602564,0.004845,0.001564,0.327127,0.905415,10.573878,0.535197
331498,6251.581818,2899.254545,13337.836364,4362.036364,0.004725,0.001836,0.327127,0.905415,10.573878,0.530968


In [22]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    

    SaveDirPath='/Data/LOBData/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')


5 194


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


373477469682.1529
path: /Data/LOBData/TrainData/Train_ORD_5_194norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__5_194norm6featNewPrc
12 100
-151679648569.0256
path: /Data/LOBData/TrainData/Train_ORD_12_100norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__12_100norm6featNewPrc
44 1
-326098198915.2143
path: /Data/LOBData/TrainData/Train_ORD_44_1norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__44_1norm6featNewPrc
17 29
60473429113.16301
path: /Data/LOBData/TrainData/Train_ORD_17_29norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__17_29norm6featNewPrc
2 83
-270959884203.92865
path: /Data/LOBData/TrainData/Train_ORD_2_83norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__2_83norm6featNewPrc
4 9997
203724722346.27368
path: /Data/LOBData/TrainData/Train_ORD_4_9997norm6featNewPrc
path: /Data/LOBData/TrainData/Train_ORD_Label__4_9997norm6featNewPrc
42 1
-1655045314960.298
path: /Data/LOBData/TrainData/Train_ORD_42_1norm6featNewPrc

In [23]:
Train_df[feat_cols].values

array([[1.32126809e+04, 1.46872787e+04, 2.62493191e+04, ...,
        8.83200393e+00, 5.35697363e-01, 1.16682500e+06],
       [2.05472722e+04, 1.33640178e+04, 3.97785344e+04, ...,
        8.83200393e+00, 5.60337045e-01, 1.12352900e+07],
       [1.29314102e+04, 1.74916449e+04, 4.11238102e+04, ...,
        8.83200393e+00, 5.48631664e-01, 1.16850000e+06],
       ...,
       [7.32241026e+03, 3.58579487e+03, 1.60867051e+04, ...,
        1.05738777e+01, 5.35196591e-01, 0.00000000e+00],
       [6.25158182e+03, 2.89925455e+03, 1.33378364e+04, ...,
        1.05738777e+01, 5.30967906e-01, 0.00000000e+00],
       [2.08184375e+03, 7.56187500e+02, 2.08184375e+03, ...,
        1.05738777e+01, 5.37776451e-01, 0.00000000e+00]])

In [24]:
Train_df[feat_cols].values.shape

(331500, 11)

In [27]:
Train_df.head(40)

,ORD_DD,ISU_CD,TM_GP,Unnamed: 0,호가장처리가격,ASK_STEP1_BSTORD_PRC,ASK_STEP1_BSTORD_RQTY,ASK_STEP2_BSTORD_PRC,ASK_STEP2_BSTORD_RQTY,ASK_STEP3_BSTORD_PRC,...,ORD_PRC,시가,고가,저가,누적체결수량,누적거래대금,직전체결가격,10단계호가합계잔량,NET_ORD_QTY,ylabel
0,20170601,KR7001840008,0.0,9.0,7092.744681,6960.276596,1439.390426,6971.287234,2453.357447,6981.510638,...,6707.010638,8.832004,0.306272,0.887197,137.373404,9.585350e+05,0.535697,53888.093617,1.166825e+06,2.0
1,20170601,KR7001840008,1.0,10.0,7120.277778,7053.033333,2520.495556,7063.355556,4947.745556,7073.355556,...,6711.266667,8.832004,0.350581,0.884147,113.202222,7.975430e+05,0.560337,68897.512222,1.123529e+07,2.0
2,20170601,KR7001840008,2.0,11.0,6986.204082,7014.836735,1181.506122,7025.285714,2255.448980,7035.346939,...,6767.469388,8.832004,0.357820,0.884147,102.765306,7.203223e+05,0.548632,81385.134694,1.168500e+06,0.0
3,20170601,KR7001840008,3.0,12.0,7111.727575,6961.727575,1038.990033,6971.727575,1040.023256,6981.727575,...,6377.707641,8.832004,0.357820,0.884147,57.893688,4.031560e+05,0.535904,69981.803987,-7.336850e+06,0.0
4,20170601,KR7001840008,4.0,13.0,7293.461538,6966.009615,970.288462,6976.009615,1629.475962,6986.009615,...,5872.884615,8.832004,0.357820,0.884147,88.134615,6.168356e+05,0.537080,81578.548077,-3.954550e+06,1.0
5,20170601,KR7001840008,5.0,14.0,7136.295265,7024.373259,1968.891365,7034.373259,5218.718663,7044.373259,...,6581.253482,8.832004,0.357820,0.884147,138.409471,9.728898e+05,0.552620,94753.768802,0.000000e+00,2.0
6,20170601,KR7001840008,6.0,15.0,7091.074074,6986.333333,655.966667,6997.777778,1024.981481,7008.037037,...,6857.555556,8.832004,0.357820,0.884147,49.181481,3.438314e+05,0.542985,108449.274074,2.726100e+05,0.0
7,20170601,KR7001840008,7.0,16.0,7075.133333,7026.466667,4063.756667,7036.700000,6039.663333,7046.700000,...,6688.766667,8.832004,0.357820,0.884147,90.436667,6.353003e+05,0.553401,108157.466667,-8.410020e+06,1.0
8,20170601,KR7001840008,8.0,17.0,7136.572438,7071.501767,5098.939929,7081.501767,11793.666078,7091.501767,...,6703.038869,8.832004,0.357820,0.884147,160.913428,1.138012e+06,0.565658,92804.314488,0.000000e+00,2.0
9,20170601,KR7001840008,9.0,18.0,7009.475983,7050.917031,765.157205,7061.310044,2158.384279,7071.310044,...,6888.034934,8.832004,0.357820,0.884147,102.152838,7.183281e+05,0.559356,91294.414847,4.304240e+06,1.0


In [26]:
Train_df['ylabel'].values

array([2., 2., 0., ..., 1., 1., 1.])

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)